In [1]:
import re
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_all = pd.read_pickle('./data/df_all_lemma.pkl')
print(len(df_all))
df_all.head(1)

2467


,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views,transcript
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110,it been great hasnt it. ive been blown away by...


In [3]:
docs_all = list(df_all['transcript'])

from nltk.corpus import stopwords

stops_standard = stopwords.words('english')
stops_custom = ['shes','youll','ill','yeah','th','yes','oh',
                'ok','okay','might','ha','mr','bg','ms',
                'mrs','ca','em','da','ted']+[re.sub('[^A-Za-z ]+', '', w) for w in stops_standard]
stop_list = list(set(stops_standard + stops_custom))

print(stop_list)

['do', "you're", 'hadnt', 'd', 'o', 'out', 'be', 'y', "you'll", 'isn', 'same', 'most', "mustn't", 'he', 'down', "didn't", 'shes', 'th', "isn't", 'it', 'both', 'ain', 'aren', 'ours', "you'd", 'and', 'weren', 'just', 'through', 'dont', 'themselves', 'of', 'we', 'ha', 'other', 'yeah', 'yourselves', 'don', "shouldn't", 'mightn', 'ill', 'ma', 'because', 'whom', 'these', 'did', "haven't", 'its', 'isnt', 'again', 'about', 'off', "don't", 'mr', 'youd', 'your', 'has', 'does', "she's", 'wasnt', 'if', 'em', 'needn', 'him', 'when', 'arent', 'mrs', 'which', 'haven', 'shan', 'own', 'their', "that'll", 'under', 'further', 'theirs', 'hasn', 'yes', 'werent', 'youll', "shan't", 'won', 'didnt', 'she', 'doing', 'against', 'youve', 'shouldnt', 'that', 'before', 'them', 'hadn', 'more', 'his', 'wouldnt', 'they', 'between', 'you', 'ourselves', 'is', "hasn't", 'by', "won't", 'at', 'on', 'all', 're', 'doesn', 'mightnt', 'didn', 'shant', 'me', 'herself', 'myself', 'into', 'a', 'few', "doesn't", 'not', 'were', 'h

In [12]:
docs = docs_all

from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.preprocessing import Normalizer


n_samples = 2000
n_features = 2000
n_components = 50
n_top_words = 10
n_gram = 1
alpha = 0.1
stop_choice= stop_list

max_df = 400
min_df = 10
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


# Use tf-idf features for LSA.
print("Extracting tf-idf features for LSA...")
tfidf_vectorizer = TfidfVectorizer(max_df=max_df, min_df=min_df,
                                   ngram_range=(n_gram,n_gram),
                                   max_features=n_features,
                                   stop_words=stop_choice)
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(docs)
print("done in %0.3fs." % (time() - t0))

# Fit the LDA model
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lsa = TruncatedSVD(n_components=n_components, algorithm = 'randomized',random_state=1)
t0 = time()
lsa.fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LSA model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(lsa, tfidf_feature_names, n_top_words)

Extracting tf-idf features for LSA...
done in 2.550s.
Fitting LDA models with tf features, n_samples=2000 and n_features=2000...
done in 0.180s.

Topics in LSA model:
Topic #0: music patient africa teacher cancer baby boy specie law india
Topic #1: cancer robot patient dna universe ocean gene specie plant molecule
Topic #2: cancer patient drug doctor tumor medical treatment hospital surgery medicine
Topic #3: music robot song universe object artist playing musician film teacher
Topic #4: robot economy device intelligence innovation leg network sensor mobile digital
Topic #5: universe galaxy star telescope particle theory physic object quantum network
Topic #6: music africa economy song growth musician china musical instrument innovation
Topic #7: universe robot music dna africa gene galaxy cancer star violence
Topic #8: dna gene genome specie virus plant bacteria genetic code molecule
Topic #9: africa african baby teacher aid plant patient hiv dna sex
Topic #10: cancer teacher tumor cl

In [13]:
X = lsa.fit_transform(tfidf)
X = Normalizer(copy=False).fit_transform(X)


In [21]:
X[1999].argmax()

13

### Find related talks

In [15]:
import sklearn.neighbors.kd_tree as kdtree

In [22]:
tree = kdtree.KDTree(X, leaf_size=10) 
N = 1999
dist, ind = tree.query([X[N]], k=20)  
for i in ind[0]:
    print(i,'\t',df_all.iloc[i].title)
#     print(i,'\t',df_all.iloc[i].views)
#     print(df_all.iloc[i].tags)

1999 	 Forget Wi-Fi. Meet the new Li-Fi Internet
1105 	 A TED speaker's worst nightmare
1873 	 How to land on a comet
373 	 A solar energy system that tracks the sun
1380 	 The mind behind Tesla, SpaceX, SolarCity ...
2423 	 You owe it to yourself to experience a total solar eclipse
2177 	 What a planet needs to sustain life
2269 	 Why Earth may someday look like Mars
2358 	 The future we're building -- and boring
1194 	 Tour the solar system from home
567 	 Fusion is energy's future
145 	 This is Saturn
1118 	 The missing link to renewable energy
1644 	 The flower-shaped starshade that might help us detect Earth-like planets
546 	 Learn to use the 13th-century astrolabe
492 	 A demo of wireless electricity
2258 	 Let's clean up the space junk orbiting Earth
961 	 Finding planets around other stars
1193 	 How to air-condition outdoor spaces
2097 	 The most mysterious star in the universe


In [23]:
df_related = pd.DataFrame(eval(df_all.iloc[N]['related_talks']))
for k in range(len(df_related)):
    print(list(df_all[df_all['title']==df_related.iloc[k]['title']].title))
#     print(df_all[df_all['title']==df_related.iloc[k]['title']].views)
#     print(list(df_all[df_all['title']==df_related.iloc[k]['title']].tags))

['Wireless data from every light bulb']
['An Internet without screens might look like this']
['Want to innovate? Become a "now-ist"']
['A solar energy system that tracks the sun']
['Caring for engineered tissue']
['A light switch for neurons']


In [12]:
# print(docs_all[395])

### Clustering using KMeans

In [509]:
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.metrics import silhouette_score
from sklearn import cluster

n_clusters = 50

# ward = cluster.AgglomerativeClustering(
#     n_clusters=n_clusters, linkage='ward',
#     connectivity=connectivity)
spectral = cluster.SpectralClustering(
    n_clusters=n_clusters, eigen_solver='arpack',
    affinity="nearest_neighbors")
Xp = spectral.fit_predict(X)
# dbscan = cluster.DBSCAN(eps=params['eps'])

# km = KMeans(n_clusters=50)
# Xt = km.fit_transform(X)

# mu_docs = km.cluster_centers_
# plt.plot(mu_docs.transpose())

# silhouette_score(X, km.labels_, metric='euclidean')

# sil_scores = []
# iner_scores = []
# ks = range(2,20)
# for k in ks:
#     km = KMeans(n_clusters=k)
#     km.fit(X)
#     sil_scores.append(silhouette_score(X, km.labels_, metric='euclidean'))
#     iner_scores.append(km.inertia_)
    
# plt.plot(ks,sil_scores)
# plt.plot(ks,iner_scores)

In [510]:
Xp.

array([18, 29, 18, ..., 28, 18, 10], dtype=int32)

In [505]:
tree = kdtree.KDTree(Xt, leaf_size=10) 
# N = 1346
dist, ind = tree.query([Xt[N]], k=10)  
for i in ind[0]:
    print(i,'\t',df_all.iloc[i].title)
#     print(i,'\t',df_all.iloc[i].views)
#     print(df_all.iloc[i].tags)

1599 	 Love -- you're doing it wrong
2343 	 3 ways to plan for the (very) long term
2085 	 How to get back to work after a career break
1395 	 What makes us feel good about our work?
1135 	 How I beat a patent troll
1872 	 How equal do we want the world to be? You'd be surprised
1821 	 The danger of hiding who you are
1870 	 5 ways to kill your dreams
1566 	 The $80 prosthetic knee that's changing lives
609 	 Plug into your hard-wired happiness


In [506]:
df_all.iloc[2343].transcript

'and theyd say great. and ive been seeing that time horizon get shorter and shorter and shorter so much so that i met with a ceo two month ago and i said we started our initial conversation. he go i love what you do. i want to talk about the next six month. we have a lot of problem that we are facing. these are civilizationalscale problem. the issue though is we cant solve them using the mental model that we use right now to try and solve these problem. yes a lot of great technical work is being done but there is a problem that we need to solve for a priori before if we want to really move the needle on those big problem. shorttermism. right. there no march. there no bracelet. there no petition that you can sign to be against shorttermism. i tried to put one up and no one signed. it wa weird. but it prevents u from doing so much. shorttermism for many reason ha pervaded every nook and cranny of our reality. i just want you to take a second and just think about an issue that youre think